# SuperHF Best of N

Initial implementation of the most basic form of Super HF:

1. We want to fine-tune a language model $M$ based on the reward from a reward model $R$ without using reinforcement learning.
1. We first get a list of prompts for our generations.
1. Iteratively, in a loop, we:
    1. Sample $p$ prompts from the training set without replacement.
    1. Use $M$ to generate $n$ completions for each prompt ($p*n$ total).
    1. Use $R$ to select the top 1 of the $n$ completions for each prompt ($p$ total).
    1. Fine-tune $M$ on the $p$ best-of-$n$ completions.
    1. Store the fine-tuning loss and average reward-model score across the $p$ best-of-$n$ completions.

# Setup

## Imports

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from superhf.data import get_superhf_prompts

## Config

In [ ]:
LANGUAGE_MODEL_NAME = "eleutherai/gpt-neo-125M"
REWARD_MODEL_NAME = "OpenAssistant/reward-model-deberta-v3-base"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Models

In [ ]:
# TODO
language_model = AutoModelForCausalLM.from_pretrained(LANGUAGE_MODEL_NAME).to(device)
reward_model = AutoModelForSequenceClassification.from_pretrained(REWARD_MODEL_NAME).to(device)

language_tokenizer = AutoTokenizer.from_pretrained(LANGUAGE_MODEL_NAME)
reward_tokenizer = AutoTokenizer.from_pretrained(REWARD_MODEL_NAME)

## Prompt dataset

In [ ]:
# TODO
dataset = get_superhf_prompts("TODO")